# Nuevo target: mediciones en el patio del IAFE - 09/06/2022

- S1: medición con placa conductora.
- S2: medición sobre la tierra del patio-

* **S_cal = S1**: señal de calibración.
* **S_target = S2**: señal reflejada por el target de interés.

> **La señal de calibración** es la que, idealmente, emite la antena. Si uno pudiese modelar de punta a punta la antena y hacer todas las cuentas necesarias, obtendría que la antena emite el pulso $S_{cal}(t)$, o bien su transformada de Fourier $\hat{S}_{cal}(f)$. Esto lo decimos porque medimos esta señal luego de la reflexión con un target que tiene el coeficiente de reflexión $|R|=1$. Está claro que esto es una hipótesis fuerte, que deberá ser corregida luego. Además, falta contemplar el factor de decaimiento de la señal debido al camino de ida-vuelta entre la antena emisora y la antena receptora. Pero este factor -geométrico- es independiente de las frecuencias del paquete de ondas y del target, así que por ahora podemos ignorarlo.


> **La señal del target** es la que debemos analizar para obtener alguna de las propiedades del blanco en estudio. En principio desconocemos la constante dieléctrica $\epsilon(\omega)$ que tiene la placa dieléctrica de espesor $D$ puesta sobre el conductor ideal. Y, en principio, la señal reflejada por este target que recibe la antena  es $S_{teo}=R(\omega)\,\times\,S_{antena}(\omega)=R(\omega)\,\times\,S_{cal}(\omega)$. Entonces, aquí tenemos que comparar $S_{teo}$ vs $S_{target}$ y ver qué tan bien andamos rumbeados. Para ello debríamos saber de antemano la constante dieléctrica del target, que sería el parámetro a obtener! Pero bueno, este es el experimento de calibración del método UWB. Así que en este caso asumimos un valor (o varios) para $\epsilon(\omega)$ y completamos el ciclo de análisis de datos.



## Skin Depth ($\delta_D$):

Dieléctrico: $\epsilon = \epsilon_r -\imath\,\epsilon_i$

### - loss-low medium:

$\delta_D = \frac{\lambda}{2\,c}\sqrt{\frac{\epsilon_r}{\epsilon^2_i}}$





## Cargamos paquetes y funciones que usaremos después ##

In [ ]:
import numpy as np
from scipy import signal
from scipy.fftpack import fft, fftfreq, fftshift
from scipy import stats

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


import os

In [ ]:
%run /home/tele/Desktop/UWB/codigos_auxiliares/to_baseband.ipynb
%run /home/tele/Desktop/UWB/codigos_auxiliares/FFT_ordenada.ipynb
%run /home/tele/Desktop/UWB/codigos_auxiliares/to_dB.ipynb
%run /home/tele/Desktop/UWB/codigos_auxiliares/mv_to_ep.ipynb
%run /home/tele/Desktop/UWB/codigos_auxiliares/R_Fresnel.ipynb

In [ ]:
## Parámetros globales de la antena

Fs = 23.328e9 #Frecuencia de muestreo [Hz] 
Ts = 1/Fs
BW = 1.5e9 

corte = 150
Nfft= 1024 # nro de frecs -> conviene que sea multiplo de 2 para calcular la FFT de manera eficiente. 
#Tiene que ser mayor a la longitud de la señal (285 para 2m, 439 para 3m)

# portadora = 'Fc7/'

if portadora== 'Fc7/':
    fc = 7.290e9
    print('fc =',str(fc/1e9) + 'GHz')
elif portadora== 'Fc8/':
    fc = 8.748e9
    print('fc =',str(fc/1e9) + 'GHz')
else:
    print('Error: portadora debe ser Fc7 o Fc8')

In [ ]:
## Selecciono la altura de la antena 

# alturas = ['1/','2/','3/','4/']

# altura = alturas[2]

## Analizo la señal de calibración - Medición 09/06/2022

In [ ]:
##----------------PREPROCESAMIENTO DE LAS SEÑALES----------------------------##

path_Scal = '/home/tele/Desktop/UWB/mediciones_09_06/S1/'

## S1/FcX/N  donde, FcX es la portadora (X=7 u 8), 1<= N <= 4 son las 4 alturas en las que tomamos mediciones.
### dentro de cada carpeta N hay 10 mediciones (1<j<10): j_S1_Ncm.txt

In [ ]:

# cuento el número de archivos en el directorio
list = os.listdir(path_Scal+portadora+altura) # dir is your directory path
N_med = int(len(list))
print('# mediciones =', N_med)
print()

# miro la longitud de cada mediciòn
L_data = np.zeros(N_med, dtype=int) # vector en donde guardo la long de cada medición


for j in range(N_med): 
    J = str(j+1)
    len_data = int(len(open(path_Scal+'Fc8/'+altura+J+'_S1.txt').readlines()))
    L_data[j] = len_data
    print('medición #'+J+' =',len_data)
    
    
print()

print('long de una medición:',np.shape(np.genfromtxt(path_Scal+portadora+altura+'1_S1.txt', delimiter=' ')))

In [ ]:
## Miro la señal en finción del tiempo

S1 = np.zeros((N_med,int(len_data-1),11),dtype=float)

# S2 = np.zeros((lendata,11,n_H))
# S2_fft = np.zeros((Nfft,n_H), dtype=complex)
# S_tar_fft = np.zeros_like(S2_fft)


for j0 in range(N_med):
    J0 = str(j0+1)
    S1[j0,:,:] = np.genfromtxt(path_Scal+portadora+altura+J0+'_S1.txt', delimiter=' ')

print('dim S1:',np.shape(S1))

## Grafico S_cal(t) en banda pasante
dist_S1 = S1[0,:,0]
tiempo_S1 = dist_S1/3e8

S1_avg  = np.mean(S1[:,:,1:],axis=2)
print('dim. S_avg =', np.shape(S1_avg))

plt.figure(figsize=(12,8))
for j in range(N_med):
    J = str(j+1)
    plt.plot(dist_S1,S1_avg[j,:],label = 'med #'+J)
plt.grid('on')
plt.legend(loc="best")
plt.show()


In [ ]:
## Recorto la señal

d_ini = 25
d_fin = 128

dist_S1 = S1[0,d_ini:d_fin,0]
tiempo_S1 = dist_S1/3e8

S1_avg  = np.mean(S1[:,d_ini:d_fin,1:],axis=2)
print('dim. S_avg =', np.shape(S1_avg))

plt.figure(figsize=(12,8))
for j in range(N_med):
    J = str(j+1)
    plt.plot(dist_S1,S1_avg[j,:],label = 'med #'+J)
plt.grid('on')
plt.legend(loc="best")
plt.show()

In [ ]:
## Transformo Fourier S_cal y grafico
Scal_fft = np.zeros((N_med,Nfft),dtype=complex)

for j in range(N_med):
    Scal_fft[j,:] = FFT_ordenada(S1_avg[j,:],Nfft,Ts)[1]
frec_fft = FFT_ordenada(S1_avg[0,:],Nfft,Ts)[0]

#promedio las 10 mediciones Scal
Scal_fft_avg = np.zeros(Nfft, dtype=complex)

for j in range(Nfft):
    Scal_fft_avg[j] = np.mean(Scal_fft[:,j])

## Grafico
fig , ax = plt.subplots(nrows=1, ncols=3, figsize=(16,6))
for j in range(N_med):
    J = str(j+1)
    ax[0].plot(frec_fft/1e9,np.real(Scal_fft[j,:]),label="med #"+ J)
    ax[0].grid('on')
    ax[0].set_xlabel('F [GHz]')
    ax[0].set_ylabel('S_cal')
    ax[0].set_title('Parte Real S_cal')
    ax[1].plot(frec_fft/1e9,np.imag(Scal_fft[j,:]),label="med #"+ J)
    ax[1].grid('on')
    ax[1].set_xlabel('F [GHz]')
    ax[1].set_ylabel('S_cal')
    ax[1].set_title('Parte Imag S_cal')
    ax[2].plot(frec_fft/1e9,np.abs(Scal_fft[j,:]),label="med #"+ J)
    ax[2].grid('on')
    ax[2].set_xlabel('F [GHz]')
    ax[2].set_ylabel('S_cal')
    ax[2].set_title('Módulo S_cal')
ax[0].plot(frec_fft/1e9,np.real(Scal_fft_avg),'-k',label = "Avg")
ax[0].legend(loc="best")
ax[1].plot(frec_fft/1e9,np.imag(Scal_fft_avg),'-k',label = "Avg")
ax[1].legend(loc="best")
ax[2].plot(frec_fft/1e9,np.abs(Scal_fft_avg),'-k',label = "Avg")
ax[2].legend(loc="best")
plt.show()

print('Scal_fft_avg:', np.shape(Scal_fft_avg))

In [ ]:
fmin = 2e9 #fc-1.55*BW
print('f_min =',fmin/1e9,'GHz')

f_pos = frec_fft[frec_fft>fmin]
np.save('fpos.npy', f_pos, allow_pickle=True, fix_imports=True)

n_frecs = len(f_pos)
start = int(1024-n_frecs)
f_norm = f_pos/1e9 # frec >0 normalizada en GHz

Scal_fpos = np.zeros((N_med,n_frecs), dtype=complex)

for j in range(N_med):
    Scal_fpos[j,:] = Scal_fft[j,start:1024]
    

Scal_avg_fpos = Scal_fft_avg[start:1024]
np.save('Scal.npy', Scal_avg_fpos, allow_pickle=True, fix_imports=True)

plt.figure(figsize=(12,8))
for j in range(N_med):
    J = str(j+1)
    plt.plot(f_pos/1e9,np.real(Scal_fpos[j,:]),label="med #"+ J)
    #plt.plot(f_pos/1e9,np.imag(Scal_fpos),label="Im(S_cal)")
    #plt.plot(f_pos/1e9,np.abs(S_cal_fpos),label="|S_cal|")
plt.plot(f_pos/1e9,np.real(Scal_avg_fpos),'+k',label="Avg")
plt.grid('on')
plt.legend(loc="best")
plt.xlabel('F [GHz]')
plt.ylabel('S_cal')
plt.show()

## Analizo las señales target ##

In [ ]:
path_Star = '/home/tele/Desktop/UWB/mediciones_09_06/S2/'

# cuento el número de archivos en el directorio
list = os.listdir(path_Star+portadora+altura) # dir is your directory path
N_med = int(len(list))
print('# mediciones =', N_med)
print()

# miro la longitud de cada mediciòn
L_data = np.zeros(N_med, dtype=int) # vector en donde guardo la long de cada medición

for j in range(N_med): 
    J = str(j+1)
    len_data_s2 = int(len(open(path_Star+'Fc8/'+altura+J+'_S2.txt').readlines()))
    L_data[j] = len_data_s2
    print('medición #'+J+' =',len_data)
    
    
print()

print('long de una medición:',np.shape(np.genfromtxt(path_Star+portadora+altura+'1_S2.txt', delimiter=' ')))

In [ ]:
## Miro la señal target en función del tiempo

S2 = np.zeros((N_med,int(len_data_s2-1),11))
print(np.shape(S2))

for j0 in range(N_med):
    J0 = str(j0+1)
    S2[j0,:,:] = np.genfromtxt(path_Star+portadora+altura+J0+'_S2.txt', delimiter=' ')



dist_S2 = S2[0,:,0]
tiempo_S2 = dist_S2/3e8

S2_avg  = np.mean(S2[:,:,1:],axis=2)
print('dim. S_avg =', np.shape(S2_avg))

plt.figure(figsize=(12,8))
for j in range(N_med):
    J = str(j+1)
    plt.plot(dist_S2,S2_avg[j,:],label = 'med #'+J)
plt.grid('on')
plt.legend(loc="best")
plt.show()

In [ ]:
## recorto la señal

dist_S2 = S2[0,d_ini:d_fin,0]
tiempo_S2 = dist_S2/3e8

S2_avg  = np.mean(S2[:,d_ini:d_fin,1:],axis=2)
print('dim. S_avg =', np.shape(S2_avg))

plt.figure(figsize=(12,8))
for j in range(N_med):
    J = str(j+1)
    plt.plot(dist_S2,S2_avg[j,:],label = 'med #'+J)
plt.grid('on')
plt.legend(loc="best")
plt.show()

In [ ]:
## Transformo Fourier S_cal y grafico
Star_fft = np.zeros((N_med,Nfft),dtype=complex)

for j in range(N_med):
    Star_fft[j,:] = FFT_ordenada(S2_avg[j,:],Nfft,Ts)[1]
frec_fft = FFT_ordenada(S2_avg[0,:],Nfft,Ts)[0]

#promedio las 10 mediciones Star
Star_fft_avg = np.zeros(Nfft, dtype=complex)

for j in range(Nfft):
    Star_fft_avg[j] = np.mean(Star_fft[:,j])

## Grafico
fig , ax = plt.subplots(nrows=1, ncols=3, figsize=(16,6))
for j in range(N_med):
    J = str(j+1)
    ax[0].plot(frec_fft/1e9,np.real(Star_fft[j,:]),label="med #"+ J)
    ax[0].grid('on')
    ax[0].set_xlabel('F [GHz]')
    ax[0].set_ylabel('S_cal')
    ax[0].set_title('Parte Real S_tar')
    ax[1].plot(frec_fft/1e9,np.imag(Star_fft[j,:]),label="med #"+ J)
    ax[1].grid('on')
    ax[1].set_xlabel('F [GHz]')
    ax[1].set_ylabel('S_cal')
    ax[1].set_title('Parte Imag S_tar')
    ax[2].plot(frec_fft/1e9,np.abs(Star_fft[j,:]),label="med #"+ J)
    ax[2].grid('on')
    ax[2].set_xlabel('F [GHz]')
    ax[2].set_ylabel('S_cal')
    ax[2].set_title('Módulo S_tar')
ax[0].plot(frec_fft/1e9,np.real(Star_fft_avg),'-.k',label = "Avg")
ax[0].legend(loc="best")
ax[1].plot(frec_fft/1e9,np.imag(Star_fft_avg),'-.k',label = "Avg")
ax[1].legend(loc="upper right")
ax[2].plot(frec_fft/1e9,np.abs(Star_fft_avg),'-.k',label = "Avg")
ax[2].legend(loc="best")
plt.show()

In [ ]:
# f_pos = frec_fft[frec_fft>fmin]
# n_frecs = len(f_pos)
# start = int(1024-n_frecs)

Star_fpos = np.zeros((N_med,n_frecs), dtype=complex)

for j in range(N_med):
    Star_fpos[j,:] = Star_fft[j,start:1024]

Star_avg_fpos = Star_fft_avg[start:1024]
np.save('Star.npy', Star_avg_fpos, allow_pickle=True, fix_imports=True)

print(np.shape(Star_avg_fpos))

plt.figure(figsize=(12,8))
for j in range(N_med):
    J = str(j+1)
    plt.plot(f_pos/1e9,np.real(Star_fpos[4,:]),label="med #"+ J)
#     #plt.plot(f_pos/1e9,np.imag(Scal_fpos),label="Im(S_cal)")
#     #plt.plot(f_pos/1e9,np.abs(S_cal_fpos),label="|S_cal|")
plt.plot(f_pos/1e9,np.real(Star_avg_fpos),'+k',label="Avg")
plt.grid('on')
plt.legend(loc="best")
plt.xlabel('F [GHz]')
plt.ylabel('S_cal')
plt.show()

## Comparo Scal vs. Star

In [ ]:
## gráfico S(t) para ambos blancos

plt.figure(figsize=(12,8))
plt.plot(dist_S2[d_ini:d_fin]/3e8,S2_avg[0,d_ini:d_fin],label = 'Target')
plt.plot(dist_S1[d_ini:d_fin]/3e8,S1_avg[0,d_ini:d_fin],label = 'Calibración')
plt.grid('on')
plt.legend(loc="best")
plt.xlabel('tiempo [nS]')
plt.title('Señal recibida en función del tiempo', fontsize=14)
plt.show()

## gráficos FFT

FFc = str(round(fc/1e9,2))

fig , ax = plt.subplots(nrows=1, ncols=3, figsize=(16,6))

ax[0].plot(f_pos/1e9,np.real(Star_avg_fpos),'-b',label = "S_tar")
ax[0].plot(f_pos/1e9,np.real(Scal_avg_fpos),'-r',label = "S_cal")
ax[0].vlines(fc/1e9,-0.3,0.3,colors='k',label = "fc ="+ FFc + "GHz")
ax[0].grid()
ax[0].set_xlabel('Frec [GHz]')
ax[0].set_ylabel('Re(S)')
ax[0].legend(loc="best")
ax[0].set_title('Parte Real')

ax[1].plot(f_pos/1e9,np.imag(Star_avg_fpos),'-b',label = "S_tar")
ax[1].plot(f_pos/1e9,np.imag(Scal_avg_fpos),'-r',label = "S_cal")
ax[1].vlines(fc/1e9,-0.2,0.3,colors='k',label = "fc ="+ FFc + "GHz")
ax[1].grid()
ax[1].set_xlabel('Frec [GHz]')
ax[1].set_ylabel('Re(S)')
ax[1].legend(loc="best")
ax[1].set_title('Parte Imaginaria')

ax[2].plot(f_pos/1e9,np.abs(Star_avg_fpos),'-b',label = "S_tar")
ax[2].plot(f_pos/1e9,np.abs(Scal_avg_fpos),'-r',label = "S_cal")
ax[2].vlines(fc/1e9,0,0.35,colors='k',label = "fc ="+ FFc + "GHz")
ax[2].grid()
ax[2].set_xlabel('Frec [GHz]')
ax[2].set_ylabel('Abs(S)')
ax[2].legend(loc="best")
ax[2].set_ylim([-1e-2,1.05*np.max(np.abs(Star_avg_fpos))])
ax[2].set_title('Módulo de las señales')
plt.show()
